In [4]:
from py_ecc.bn128 import G1, G2, multiply, add, curve_order, Z1, pairing, neg, final_exponentiate, FQ12, is_inf, eq
import galois 
GF = galois.GF(curve_order)

In [9]:
# This script demonstrates the poly evaluations with powers of tau
# we see that evaluating a polyomial at tau is the same as computing the inner product of the poly coefficients and the powers of tau
tau = GF(20)


# unencrypted evaluation
# Create polynomial 4x^4 + 10x^3 + 7x^2 + 7x + 2
# evaluate at tau = 20 i.e. 4(20)^4 + 10(20)^3 + 7(20)^2 + 7(20) + 2 = 722942

poly = galois.Poly([4,2,3,1], field=GF) * galois.Poly([1,2], field=GF)
# poly = 4x^4 + 10x^3 + 7x^2 + 7x + 2

# Evaluate the poly at tau.
unencrypted_p_tau = poly(tau)
# = 722942

unencrypted_p_tau_G1 = multiply(G1, int(unencrypted_p_tau))
# = 722942 * G1




# encrypted evaluation
# generate powers of tau (1 for each coeff) for 4x^4 + 10x^3 + 7x^2 + 7x + 2
# i.e. [τ^4 G1],[τ^3 G1],[τ^2 G1],[τ^1 G1],[τ^0 G1]
# inner product powers of tau with poly coeffs
# i.e. 4 * [τ^4 G1] + 10 * [τ^3 G1] + 7 * [τ^2 G1] + 7 * [τ^1 G1] + 2 * [τ^0 G1]
# result should be equal to 

def generate_powers_of_tau(tau, degree):
    """Computes and prints the powers of tau up to the specified degree and returns them
    as elliptic curve points by multiplying each power by the base point G1."""
    powers_of_tau = []
    for i in range(degree+1):
        powers_of_tau.append(multiply(G1, int(tau**i)))
    return powers_of_tau

# create [τ^0 G1],[τ^1 G1][τ^2 G1]...[τ^degree +1 G1]
powers_of_tau = generate_powers_of_tau(tau, poly.degree)

def inner_product(powers_of_tau, coeffs):
    """Computes the elliptic curve point representing the inner product of the reversed poly coefficients
    and the powers of tau points."""
    accumulator = Z1
    for tau_power, coeff in zip(powers_of_tau, coeffs):
        term = multiply(tau_power, int(coeff))
        accumulator = add(accumulator, term)
    return accumulator

encrypted_p_tau_G1 = inner_product(powers_of_tau, poly.coeffs[::-1])

# test that the encrypted = unencrypted
assert eq(encrypted_p_tau_G1, unencrypted_p_tau_G1), "doesn't match"


# test random pairing
# pairing requires powers e(a * b) = e()^ab
# let lhs = (10, 5)... let rhs = (50,1)
lhs = pairing(
    multiply(G2,10),
    multiply(G1, 5)
)

rhs = pairing(
    multiply(G2,50),
    multiply(G1,1)
)

assert eq(lhs,rhs), "not equal"